In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
import math
from datetime import datetime

LOG_FILE = "risultati_equazioni_secondo_grado.txt"

# ------------------------ Tema / Stile (chiaro) ------------------------

def apply_style_light(root: tk.Tk):
    style = ttk.Style(root)
    try:
        style.theme_use("clam")
    except tk.TclError:
        pass

    palette = {
        "bg": "#F6F8FC",
        "card": "#FFFFFF",
        "card2": "#F1F5F9",
        "text": "#0F172A",
        "muted": "#475569",
        "border": "#D6DEE8",
        "accent": "#2563EB",
        "accent_hover": "#1D4ED8",
        "danger": "#DC2626",
        "danger_hover": "#B91C1C",
        "btn": "#EAF0FF",
        "btn_hover": "#DCE7FF",
        "input_bg": "#FFFFFF",
        "success": "#16A34A",
    }

    # Font moderni (fallback automatico in base al sistema)
    fonts = {
        "title": ("Segoe UI", 16, "bold"),
        "subtitle": ("Segoe UI", 10),
        "label": ("Segoe UI", 10),
        "btn": ("Segoe UI", 10, "bold"),
        "mono": ("Cascadia Mono", 10),
        "mono_big": ("Cascadia Mono", 11),
        "out_mono": ("Cascadia Mono", 11),
        "steps": ("Cascadia Mono", 11),
    }

    root.configure(bg=palette["bg"])

    style.configure("App.TFrame", background=palette["bg"])

    style.configure("Title.TLabel", background=palette["bg"], foreground=palette["text"], font=fonts["title"])
    style.configure("Sub.TLabel", background=palette["bg"], foreground=palette["muted"], font=fonts["subtitle"])
    style.configure("Field.TLabel", background=palette["card"], foreground=palette["text"], font=fonts["label"])

    style.configure("TLabelFrame", background=palette["card"], foreground=palette["text"], bordercolor=palette["border"])
    style.configure("TLabelframe.Label", background=palette["card"], foreground=palette["text"],
                    font=("Segoe UI", 10, "bold"))

    style.configure(
        "TEntry",
        fieldbackground=palette["input_bg"],
        foreground=palette["text"],
        insertcolor=palette["accent"],
        bordercolor=palette["border"],
        lightcolor=palette["border"],
        darkcolor=palette["border"]
    )

    style.configure(
        "TButton",
        background=palette["btn"],
        foreground=palette["text"],
        bordercolor=palette["border"],
        focusthickness=2,
        focuscolor=palette["accent"],
        font=fonts["btn"],
        padding=(12, 8)
    )
    style.map(
        "TButton",
        background=[("active", palette["btn_hover"]), ("pressed", palette["btn_hover"])],
        foreground=[("disabled", "#94A3B8")]
    )

    style.configure("Accent.TButton", background=palette["accent"], foreground="#FFFFFF", bordercolor=palette["accent"])
    style.map("Accent.TButton", background=[("active", palette["accent_hover"]), ("pressed", palette["accent_hover"])])

    style.configure("Danger.TButton", background=palette["danger"], foreground="#FFFFFF", bordercolor=palette["danger"])
    style.map("Danger.TButton", background=[("active", palette["danger_hover"]), ("pressed", palette["danger_hover"])])

    style.configure("Ghost.TButton", background="#FFFFFF", foreground=palette["text"], bordercolor=palette["border"])
    style.map("Ghost.TButton", background=[("active", "#F8FAFC"), ("pressed", "#F1F5F9")])

    return palette, fonts


# ------------------------ Utility ------------------------

def parse_float(s: str) -> float:
    s = s.strip().replace(",", ".")
    return float(s)

def format_number(x: float, ndigits: int = 10) -> str:
    if abs(x) < 1e-12:
        x = 0.0
    return f"{x:.{ndigits}g}"

def salva_su_file(testo: str):
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write("=" * 60 + "\n")
        f.write(f"Data e ora: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n")
        f.write(testo)
        f.write("\n")


# ------------------------ Stato applicazione ------------------------

last_solution = {
    "valid": False,
    "a": None, "b": None, "c": None,
    "delta": None,
    "roots_real": None,   # None oppure lista [x] o [x1, x2]
    "steps": ""           # passaggi di calcolo (testo)
}


# ------------------------ Logica risoluzione + passaggi ------------------------

def build_steps(a: float, b: float, c: float, delta: float, roots_real):
    """
    Costruisce una spiegazione testuale dei passaggi:
    - formula del discriminante
    - calcolo numerico di Δ
    - formula risolutiva e sostituzione
    """
    lines = []
    lines.append("PASSAGGI DI CALCOLO\n")
    lines.append("1) Discriminante")
    lines.append("   Δ = b² − 4ac")
    lines.append(f"   Δ = ({format_number(b)})² − 4·({format_number(a)})·({format_number(c)})")
    lines.append(f"   Δ = {format_number(b*b)} − {format_number(4*a*c)}")
    lines.append(f"   Δ = {format_number(delta)}")
    lines.append("")
    lines.append("2) Formula risolutiva")
    lines.append("   x = (−b ± √Δ) / (2a)")
    lines.append(f"   a = {format_number(a)}, b = {format_number(b)}, c = {format_number(c)}")
    lines.append("")

    eps = 1e-12
    if delta > eps:
        sqrt_d = math.sqrt(delta)
        x1 = roots_real[0]
        x2 = roots_real[1]
        lines.append("   Caso Δ > 0: due soluzioni reali distinte")
        lines.append(f"   √Δ = √({format_number(delta)}) = {format_number(sqrt_d)}")
        lines.append(f"   x₁ = (−({format_number(b)}) + {format_number(sqrt_d)}) / (2·{format_number(a)})")
        lines.append(f"   x₁ = {format_number(x1)}")
        lines.append(f"   x₂ = (−({format_number(b)}) − {format_number(sqrt_d)}) / (2·{format_number(a)})")
        lines.append(f"   x₂ = {format_number(x2)}")
    elif abs(delta) <= eps:
        x = roots_real[0]
        lines.append("   Caso Δ = 0: una soluzione reale doppia")
        lines.append(f"   √Δ = √({format_number(delta)}) = 0")
        lines.append(f"   x = (−({format_number(b)})) / (2·{format_number(a)})")
        lines.append(f"   x = {format_number(x)}")
    else:
        # Per completezza: radici complesse con i = √−1
        sqrt_abs = math.sqrt(-delta)
        real = (-b) / (2*a)
        imag = sqrt_abs / (2*abs(a))
        if a < 0:
            imag = -imag
        lines.append("   Caso Δ < 0: nessuna soluzione reale (soluzioni complesse coniugate)")
        lines.append(f"   √Δ = √({format_number(delta)}) = i·√({format_number(-delta)})")
        lines.append(f"   √(|Δ|) = √({format_number(-delta)}) = {format_number(sqrt_abs)}")
        lines.append("   x = (−b ± i·√(|Δ|)) / (2a)")
        lines.append(f"   parte reale:  −b/(2a) = {format_number(real)}")
        lines.append(f"   parte immaginaria: √(|Δ|)/(2a) = {format_number(imag)}")
        lines.append(f"   x₁ = {format_number(real)} + {format_number(imag)}i")
        lines.append(f"   x₂ = {format_number(real)} - {format_number(imag)}i")

    return "\n".join(lines) + "\n"

def solve():
    display.config(state="normal")
    display.delete("1.0", "end")

    last_solution["valid"] = False
    last_solution["steps"] = ""
    btn_plot.config(state="disabled")
    btn_steps.config(state="disabled")

    try:
        a = parse_float(entry_a.get())
        b = parse_float(entry_b.get())
        c = parse_float(entry_c.get())
    except ValueError:
        display.tag_configure("err", foreground=PALETTE["danger"])
        display.insert("end", "ERRORE: inserire valori numerici validi per a, b, c.\n", "err")
        display.config(state="disabled")
        return

    if abs(a) < 1e-12:
        display.tag_configure("err", foreground=PALETTE["danger"])
        display.insert("end", "ERRORE: a = 0 ⇒ non è un’equazione di secondo grado.\n", "err")
        display.config(state="disabled")
        return

    delta = b*b - 4*a*c

    output = ""
    output += "================ RISULTATO ================\n"
    output += f"Equazione:  {format_number(a)}·x²  +  {format_number(b)}·x  +  {format_number(c)}  =  0\n"
    output += f"Δ = b² − 4ac = {format_number(delta)}\n"
    output += "-------------------------------------------\n"

    eps = 1e-12
    roots_real = None

    if delta > eps:
        sqrt_d = math.sqrt(delta)
        x1 = (-b + sqrt_d) / (2*a)
        x2 = (-b - sqrt_d) / (2*a)
        roots_real = [x1, x2]
        output += "Caso: Δ > 0  →  due soluzioni reali distinte\n"
        output += f"x₁ = {format_number(x1)}\n"
        output += f"x₂ = {format_number(x2)}\n"
    elif abs(delta) <= eps:
        x = (-b) / (2*a)
        roots_real = [x]
        output += "Caso: Δ = 0  →  una soluzione reale doppia\n"
        output += f"x  = {format_number(x)}\n"
    else:
        real = (-b) / (2*a)
        imag = math.sqrt(-delta) / (2*abs(a))
        if a < 0:
            imag = -imag
        output += "Caso: Δ < 0  →  due soluzioni complesse coniugate\n"
        output += f"x₁ = {format_number(real)}  +  {format_number(imag)}i\n"
        output += f"x₂ = {format_number(real)}  -  {format_number(imag)}i\n"

    output += "===========================================\n"

    display.insert("end", output)
    display.config(state="disabled")

    # Salvataggio SOLO dei risultati corretti
    salva_su_file(output)

    # Passaggi di calcolo (solo se risoluzione valida)
    steps_text = build_steps(a, b, c, delta, roots_real)

    last_solution.update({
        "valid": True,
        "a": a, "b": b, "c": c,
        "delta": delta,
        "roots_real": roots_real,
        "steps": steps_text
    })

    btn_plot.config(state="normal")
    btn_steps.config(state="normal")

def clear_all():
    entry_a.delete(0, "end")
    entry_b.delete(0, "end")
    entry_c.delete(0, "end")
    display.config(state="normal")
    display.delete("1.0", "end")
    display.config(state="disabled")

    last_solution["valid"] = False
    last_solution["steps"] = ""
    btn_plot.config(state="disabled")
    btn_steps.config(state="disabled")
    entry_a.focus_set()

def mostra_cronologia():
    win = tk.Toplevel(root)
    win.title("Cronologia equazioni risolte")
    win.minsize(820, 480)
    win.configure(bg=PALETTE["bg"])

    container = ttk.Frame(win, style="App.TFrame", padding=16)
    container.grid(row=0, column=0, sticky="nsew")
    win.grid_rowconfigure(0, weight=1)
    win.grid_columnconfigure(0, weight=1)

    ttk.Label(container, text="Cronologia", style="Title.TLabel").grid(row=0, column=0, sticky="w")
    ttk.Label(container, text=f"File: {LOG_FILE}", style="Sub.TLabel").grid(row=1, column=0, sticky="w", pady=(2, 12))

    card = ttk.Frame(container, style="TFrame", padding=0)
    card.grid(row=2, column=0, sticky="nsew")
    container.grid_rowconfigure(2, weight=1)
    card.grid_rowconfigure(0, weight=1)
    card.grid_columnconfigure(0, weight=1)

    txt = tk.Text(
        card,
        wrap="word",
        bg="#FFFFFF",
        fg=PALETTE["text"],
        insertbackground=PALETTE["accent"],
        relief="flat",
        font=FONTS["out_mono"]
    )
    txt.grid(row=0, column=0, sticky="nsew")

    scroll = ttk.Scrollbar(card, orient="vertical", command=txt.yview)
    scroll.grid(row=0, column=1, sticky="ns", padx=(10, 0))
    txt.config(yscrollcommand=scroll.set)

    try:
        with open(LOG_FILE, "r", encoding="utf-8") as f:
            contenuto = f.read().strip()
        if not contenuto:
            contenuto = "Cronologia vuota: nessuna equazione salvata."
    except FileNotFoundError:
        contenuto = "File di cronologia non trovato. Nessuna equazione è stata ancora salvata."
    except OSError as e:
        contenuto = f"Impossibile leggere la cronologia.\nDettagli: {e}"

    txt.insert("1.0", contenuto)
    txt.config(state="disabled")

def cancella_cronologia():
    ok = messagebox.askyesno(
        "Conferma cancellazione",
        "Cancellare definitivamente la cronologia?\n"
        "Verrà eliminato anche il contenuto del file di testo."
    )
    if not ok:
        return

    try:
        with open(LOG_FILE, "w", encoding="utf-8") as f:
            f.write("")
        messagebox.showinfo("Operazione completata", "Cronologia cancellata correttamente.")
    except OSError as e:
        messagebox.showerror("Errore", f"Impossibile cancellare la cronologia.\nDettagli: {e}")

def show_steps():
    if not last_solution["valid"] or not last_solution["steps"]:
        messagebox.showwarning("Passaggi", "Nessuna equazione valida disponibile. Eseguire prima “Risolvi”.")
        return

    win = tk.Toplevel(root)
    win.title("Passaggi di calcolo")
    win.minsize(820, 520)
    win.configure(bg=PALETTE["bg"])

    container = ttk.Frame(win, style="App.TFrame", padding=16)
    container.grid(row=0, column=0, sticky="nsew")
    win.grid_rowconfigure(0, weight=1)
    win.grid_columnconfigure(0, weight=1)

    ttk.Label(container, text="Procedimento di risoluzione", style="Title.TLabel").grid(row=0, column=0, sticky="w")
    a = last_solution["a"]; b = last_solution["b"]; c = last_solution["c"]
    ttk.Label(
        container,
        text=f"Equazione: {format_number(a)}·x² + {format_number(b)}·x + {format_number(c)} = 0",
        style="Sub.TLabel"
    ).grid(row=1, column=0, sticky="w", pady=(2, 12))

    card = ttk.Frame(container, padding=0)
    card.grid(row=2, column=0, sticky="nsew")
    container.grid_rowconfigure(2, weight=1)
    card.grid_rowconfigure(0, weight=1)
    card.grid_columnconfigure(0, weight=1)

    txt = tk.Text(
        card,
        wrap="word",
        bg="#FFFFFF",
        fg=PALETTE["text"],
        insertbackground=PALETTE["accent"],
        relief="flat",
        font=FONTS["steps"]
    )
    txt.grid(row=0, column=0, sticky="nsew")

    scroll = ttk.Scrollbar(card, orient="vertical", command=txt.yview)
    scroll.grid(row=0, column=1, sticky="ns", padx=(10, 0))
    txt.config(yscrollcommand=scroll.set)

    txt.insert("1.0", last_solution["steps"])
    txt.config(state="disabled")

def on_enter(_event=None):
    solve()


# ------------------------ Grafico (Canvas) ------------------------

def f(a, b, c, x):
    return a*x*x + b*x + c

def plot_parabola():
    if not last_solution["valid"]:
        messagebox.showwarning("Grafico", "Nessuna equazione valida disponibile. Eseguire prima “Risolvi”.")
        return

    a = last_solution["a"]
    b = last_solution["b"]
    c = last_solution["c"]
    roots = last_solution["roots_real"]

    win = tk.Toplevel(root)
    win.title("Grafico della parabola")
    win.minsize(980, 600)
    win.configure(bg=PALETTE["bg"])

    outer = ttk.Frame(win, style="App.TFrame", padding=16)
    outer.grid(row=0, column=0, sticky="nsew")
    win.grid_rowconfigure(0, weight=1)
    win.grid_columnconfigure(0, weight=1)

    ttk.Label(
        outer,
        text=f"f(x) = {format_number(a)}·x² + {format_number(b)}·x + {format_number(c)}",
        style="Title.TLabel"
    ).grid(row=0, column=0, sticky="w")

    roots_txt = "nessuna (soluzioni complesse)" if roots is None else ", ".join(format_number(r) for r in roots)
    ttk.Label(outer, text=f"Intersezioni con l’asse x: {roots_txt}", style="Sub.TLabel").grid(
        row=1, column=0, sticky="w", pady=(2, 12)
    )

    card = ttk.Frame(outer, padding=12)
    card.grid(row=2, column=0, sticky="nsew")
    outer.grid_rowconfigure(2, weight=1)
    card.grid_rowconfigure(0, weight=1)
    card.grid_columnconfigure(0, weight=1)

    W, H = 920, 440
    cvs = tk.Canvas(
        card,
        width=W,
        height=H,
        bg="#FFFFFF",
        highlightthickness=1,
        highlightbackground=PALETTE["border"]
    )
    cvs.grid(row=0, column=0, sticky="nsew")

    xv = -b / (2*a)
    if roots is not None and len(roots) == 2:
        x_min = min(roots) - 2.0
        x_max = max(roots) + 2.0
    elif roots is not None and len(roots) == 1:
        x_min = roots[0] - 4.0
        x_max = roots[0] + 4.0
    else:
        x_min = xv - 6.0
        x_max = xv + 6.0

    if abs(x_max - x_min) < 1e-9:
        x_min -= 5.0
        x_max += 5.0

    N = 800
    xs = [x_min + (x_max - x_min) * i / (N - 1) for i in range(N)]
    ys = [f(a, b, c, x) for x in xs]
    y_min = min(ys)
    y_max = max(ys)
    if abs(y_max - y_min) < 1e-9:
        y_min -= 1.0
        y_max += 1.0

    pad = 58
    gx0, gy0 = pad, pad
    gx1, gy1 = W - pad, H - pad

    def x_to_px(x):
        return gx0 + (x - x_min) * (gx1 - gx0) / (x_max - x_min)

    def y_to_py(y):
        return gy1 - (y - y_min) * (gy1 - gy0) / (y_max - y_min)

    grid_col = "#EEF2F7"
    axis_col = PALETTE["text"]
    box_col = PALETTE["border"]

    cvs.create_rectangle(gx0, gy0, gx1, gy1, outline=box_col)

    steps = 10
    for i in range(1, steps):
        xg = gx0 + i * (gx1 - gx0) / steps
        yg = gy0 + i * (gy1 - gy0) / steps
        cvs.create_line(xg, gy0, xg, gy1, fill=grid_col)
        cvs.create_line(gx0, yg, gx1, yg, fill=grid_col)

    if y_min <= 0 <= y_max:
        y0 = y_to_py(0.0)
        cvs.create_line(gx0, y0, gx1, y0, fill=axis_col, width=2)
        cvs.create_text(gx1, y0, text="  y=0", anchor="w", fill=axis_col, font=("Segoe UI", 9, "bold"))
    if x_min <= 0 <= x_max:
        x0 = x_to_px(0.0)
        cvs.create_line(x0, gy0, x0, gy1, fill=axis_col, width=2)
        cvs.create_text(x0, gy0 - 8, text="x=0", anchor="s", fill=axis_col, font=("Segoe UI", 9, "bold"))

    lbl_font = ("Segoe UI", 9)
    cvs.create_text(gx0, gy1 + 22, text=format_number(x_min), anchor="w", fill=PALETTE["muted"], font=lbl_font)
    cvs.create_text(gx1, gy1 + 22, text=format_number(x_max), anchor="e", fill=PALETTE["muted"], font=lbl_font)
    cvs.create_text(gx0 - 10, gy1, text=format_number(y_min), anchor="e", fill=PALETTE["muted"], font=lbl_font)
    cvs.create_text(gx0 - 10, gy0, text=format_number(y_max), anchor="e", fill=PALETTE["muted"], font=lbl_font)

    curve_col = PALETTE["accent"]
    pts = []
    for x, y in zip(xs, ys):
        pts.extend([x_to_px(x), y_to_py(y)])
    cvs.create_line(*pts, width=3, fill=curve_col, smooth=True)

    yv = f(a, b, c, xv)
    vx, vy = x_to_px(xv), y_to_py(yv)
    r = 5
    cvs.create_oval(vx - r, vy - r, vx + r, vy + r, outline="", fill=axis_col)
    cvs.create_text(
        vx + 10, vy - 12,
        text=f"V({format_number(xv)}, {format_number(yv)})",
        anchor="w", fill=axis_col, font=("Segoe UI", 9, "bold")
    )

    if roots is not None:
        root_col = PALETTE["success"]
        for i, xr in enumerate(roots, start=1):
            px = x_to_px(xr)
            py = y_to_py(0.0) if (y_min <= 0 <= y_max) else y_to_py(f(a, b, c, xr))
            rr = 5
            cvs.create_oval(px - rr, py - rr, px + rr, py + rr, outline="", fill=root_col)
            cvs.create_text(
                px, py + 16,
                text=f"x{i} = {format_number(xr)}",
                anchor="n", fill=root_col, font=("Segoe UI", 9, "bold")
            )


# ------------------------ GUI ------------------------

root = tk.Tk()
root.title("Risolutore Equazione di Secondo Grado")
root.resizable(False, False)

PALETTE, FONTS = apply_style_light(root)

outer = ttk.Frame(root, style="App.TFrame", padding=18)
outer.grid(row=0, column=0, sticky="nsew")

ttk.Label(outer, text="Risolutore di Equazioni di Secondo Grado", style="Title.TLabel").grid(
    row=0, column=0, sticky="w"
)
ttk.Label(
    outer,
    text="Inserire i coefficienti (a, b, c) e premere “Risolvi”.  Invio = Risolvi",
    style="Sub.TLabel"
).grid(row=1, column=0, sticky="w", pady=(2, 14))

card_in = ttk.LabelFrame(outer, text="Coefficienti (ax² + bx + c = 0)", padding=(12, 10))
card_in.grid(row=2, column=0, sticky="ew")
card_in.grid_columnconfigure(1, weight=1)

ttk.Label(card_in, text="a", style="Field.TLabel").grid(row=0, column=0, sticky="e", padx=(0, 10), pady=6)
entry_a = ttk.Entry(card_in, width=24, justify="right")
entry_a.grid(row=0, column=1, sticky="w", pady=6)

ttk.Label(card_in, text="b", style="Field.TLabel").grid(row=1, column=0, sticky="e", padx=(0, 10), pady=6)
entry_b = ttk.Entry(card_in, width=24, justify="right")
entry_b.grid(row=1, column=1, sticky="w", pady=6)

ttk.Label(card_in, text="c", style="Field.TLabel").grid(row=2, column=0, sticky="e", padx=(0, 10), pady=6)
entry_c = ttk.Entry(card_in, width=24, justify="right")
entry_c.grid(row=2, column=1, sticky="w", pady=6)

btn_row = ttk.Frame(outer, style="App.TFrame")
btn_row.grid(row=3, column=0, sticky="w", pady=(14, 12))

btn_solve = ttk.Button(btn_row, text="Risolvi", style="Accent.TButton", command=solve)
btn_solve.grid(row=0, column=0, padx=(0, 8))

btn_clear = ttk.Button(btn_row, text="C", style="Ghost.TButton", command=clear_all, width=5)
btn_clear.grid(row=0, column=1, padx=(0, 12))

btn_history = ttk.Button(btn_row, text="Cronologia", command=mostra_cronologia)
btn_history.grid(row=0, column=2, padx=(0, 8))

btn_del_history = ttk.Button(btn_row, text="Cancella cronologia", style="Danger.TButton", command=cancella_cronologia)
btn_del_history.grid(row=0, column=3, padx=(0, 12))

btn_plot = ttk.Button(btn_row, text="Grafico", command=plot_parabola)
btn_plot.grid(row=0, column=4, padx=(0, 8))
btn_plot.config(state="disabled")

# Nuovo pulsante: passaggi di calcolo
btn_steps = ttk.Button(btn_row, text="Passaggi", command=show_steps)
btn_steps.grid(row=0, column=5)
btn_steps.config(state="disabled")

card_out = ttk.LabelFrame(outer, text="Output", padding=(12, 10))
card_out.grid(row=4, column=0, sticky="ew")

out_wrap = ttk.Frame(card_out)
out_wrap.grid(row=0, column=0, sticky="ew")
out_wrap.grid_columnconfigure(0, weight=1)

display = tk.Text(
    out_wrap,
    width=88,
    height=12,
    wrap="word",
    bg="#FFFFFF",
    fg=PALETTE["text"],
    insertbackground=PALETTE["accent"],
    relief="flat",
    font=FONTS["out_mono"]
)
display.grid(row=0, column=0, sticky="nsew")

scroll_out = ttk.Scrollbar(out_wrap, orient="vertical", command=display.yview)
scroll_out.grid(row=0, column=1, sticky="ns", padx=(10, 0))
display.config(yscrollcommand=scroll_out.set, state="disabled")

root.bind("<Return>", on_enter)
entry_a.focus_set()

root.mainloop()
